In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("bci05.csv")
data.head()

,tag,sp,gx,gy,dbh,pom,date,codes,status
0,105951,ACACME,610.0,104.7,119.0,1,8924.0,M,A
1,132160,ACACME,534.8,241.3,116.0,1,8922.0,*,A
2,132234,ACACME,539.4,242.3,NaN,0,8922.0,DN,D
3,132235,ACACME,538.8,242.5,NaN,0,8922.0,DN,D
4,191542,ACACME,282.7,177.5,75.0,1,8825.0,*,A


In [3]:
data = data[data["status"]=='A'][["tag", "sp", "gx", "gy"]]

In [4]:
# 1) Species spotting
types = data['sp'].value_counts().keys()
print("There are {} different species".format(len(types)))

There are 299 different species


In [6]:
# 2) Subsampling
# 50 * 50 meters
data["i"], data["j"] = data["gx"]//50, data["gy"]//50
# a)
cell_pop = data.groupby(["i", "j"])["sp"].value_counts()
# b)
cell_pop_M = cell_pop.unstack().stack(dropna=False).fillna(0).astype(int)
cell_pop_M = np.array(cell_pop_M).reshape(20,10, 299) # i, j, specie
cell_pop_fraqs = np.mean(cell_pop_M / cell_pop_M.sum(axis=2)[:,:,np.newaxis], axis=(0,1))
print("And of course the sum of the normalized frequencies is: ", round(cell_pop_fraqs.sum()))

And of course the sum of the normalized frequencies is:  1.0


In [10]:
# 3)


SyntaxError: invalid syntax (<ipython-input-10-a4d268a5b222>, line 2)